In [26]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/sam11/Git/ml-py


### Greystone

In [8]:
user = "neo4j"
password = 'GREYSTONE!!'

csv_file = os.path.join(project_path ,'data', 'gs_data', 'gs_tool_tag_export.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'gs_data', 'gs_header.yaml')

### Mine

In [13]:
user = "neo4j"
password = 'MINE!!'

csv_file = os.path.join(project_path ,'data', 'public', 'READABLE_TAGS.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'public', 'mine_header.yaml')

### HVAC

In [2]:
user = "neo4j"
password = 'HVAC!!'

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

### PSU

In [27]:
user = "neo4j"
password = 'PSU!!'

csv_file = os.path.join(project_path ,'data', 'psu_data', 'psu_data.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'psu_data', 'psu_header.yaml')

### Auto

In [20]:
user = "neo4j"
password = 'AUTO!!'

csv_file = os.path.join(project_path ,'data', 'auto_data', 'auto_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'auto_data', 'auto_header.yaml')

### Others

In [ ]:
user = getpass.getuser()
password = getpass.getpass("Password:")

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

## Connect to DB

In [28]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


## Load CSV

In [29]:
dataframe = pd.read_csv(csv_file)
dataframe.fillna("", inplace=True)

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


### Clean the database

In [30]:
database.deleteData()
database.dropIndexes()
database.createIndexes()

1

## Store the data

In [31]:
done = graphDatabase_from_TaggedCSV(database, dataframe, csvSchemas_dict)

100%|██████████| 13416/13416 [01:26<00:00, 155.13it/s]


# KPI

## Create the object for the KPI

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *

databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)

In [ ]:
m = MachineKpi(operator="=", name = "h1", databaseInfo=databaseSchema_dict)
t = TechnicianKpi(operator=".-", name="steve", result="name", databaseInfo=databaseSchema_dict)
i = TagItemKpi(operator="<>", keyword="valve", result="keyword", databaseInfo=databaseSchema_dict)

In [ ]:
kpi = m+t+i
print(kpi.cypher_createQuery())

In [ ]:
done, filterResult = database.runQuery(kpi.cypher_createQuery())

dataframe = resultToObservationDataframe(filterResult)
dataframe